# Stockformer Demo

In [ ]:
import sys
# if not 'Informer2020' in sys.path:
#     sys.path += ['Informer2020']

## Experiments: Train and Test

In [ ]:
from utils.tools import dotdict
from exp.exp_informer import Exp_Informer
import torch
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from utils.ipynb_helpers import args_from_setting, setting_from_args, handle_gpu

In [ ]:
args = dotdict()
args.des = 'full_1h'

args.model = 'mlp' # 'stockformer'

args.data = 'custom' # data
args.checkpoints = './checkpoints' # location of model checkpoints
args.root_path = './data/ETT/' # root path of data file

args.data_path = 'full_1h.csv' # data file
args.freq = 'h' # freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h

args.features = 'MS' # forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate
args.target = 'XOM_pctchange' # target feature in S or MS task


args.seq_len = 1 # input sequence length of Informer encoder
args.label_len = 1 # start token length of Informer decoder
args.pred_len = 1 # prediction sequence length

#["XOM_close", "BP_close", "CVX_close", "WTI_close"]
#["XOM_open", "XOM_high", "XOM_low", "XOM_close", "XOM_volume", "XOM_pctchange", "XOM_shortsma"]
args.cols = ["XOM_open", "XOM_close", "XOM_pctchange", "XOM_shortsma", 
                'CVX_pctchange', 'COP_pctchange', 'BP_pctchange', 'PBR_pctchange', 
                'WTI_pctchange', 'EOG_pctchange', 'ENB_pctchange', 'SLB_pctchange',
                ]#'C:USDSAR_pctchange'

args.enc_in = len(args.cols) # encoder input size
args.dec_in = len(args.cols) # decoder input size # TODO: Remove
args.c_out = 1 if args.features in ["S", "MS"] else args.dec_in # output size


args.d_model = 128 # dimension of model; this is also the dimension of the token embeddings
args.n_heads = 8 # num of attention heads
args.e_layers = 4 # num of encoder layers
args.d_layers = 4 # num of decoder layers # TODO: Remove
args.d_ff = 2048 # dimension of fcn in model
args.dropout = 0.5 # dropout
args.embed = None#'timeF' # time features encoding, options:[timeF, fixed, learned, None]
args.activation = 'gelu' # activation

args.attn = 'full' # attention used in encoder, options:[prob, full]
args.factor = 5 # probsparse attn factor; doesn't matter unless args.attn==full
args.distil = False # whether to use distilling in encoder
args.output_attention = False # whether to output attention in encoder
args.mix = False # whether to use mixed attention
args.padding = 0 # TODO: Remove

args.batch_size = 128 #64
args.learning_rate = 0.001
args.loss = 'mse'
args.lradj = "type3" # What learning rate scheduler to use: ["type3", None, "type1"]
args.train_epochs = 2
args.patience = 30 # For early stopping

args.use_amp = False # whether to use automatic mixed precision training
args.num_workers = 0
args.itr = 1 # number of runs

args.scale = True # whether to scale to mean 0, var 1
args.inverse = False # whether to invert that scale before loss is calculated, lets keep this at False

# This is for debugging to overfit
# When True, patience doesn't matter at all and the model-state that is saved is the one after the last epoch
# When False, the model-state that is saved is the one with the highest validation-loss and we can early stop with patience
args.no_early_stop = True 


# Control data split from args, either a date string like "2000-01-30" or None (for default)
args.date_start = "2012-01-01" # Train data starts on this date, default is to go back as far as possible
args.date_end = "2020-01-01" # Train data starts on this date, default is to go back as far as possible
args.date_test = "2019-06-01" # Test data is data after this date, default is to use ~20% of the data as test data



#args.load_model_path = "stockformer_custom_ftMS_sl16_ll4_pl1_ei12_di12_co1_iFalse_dm512_nh8_el12_dl4_df2048_atfull_fc5_ebtimeF_dtFalse_mxFalse_pretrain_full_1h_0/checkpoint-pretrain.pth"

# Code to handle gpu
# None to use all available GPUs
# False for not using GPUs, 
# 0 for using cuda:0, 
# "0,1" for using both cuda:0 and cuda:1
handle_gpu(args, None)

# TODO: Figure out what this is for
args.detail_freq = args.freq
args.freq = args.freq[-1:]


print('Args in experiment:')
print(args)
Exp = Exp_Informer


# import json
# with open("configs/no_hist.json", "w") as f:
#     json.dump(args, f, indent=4)

### Train & Test *args.itr* models

In [ ]:
exp = None
setting = None
for ii in range(args.itr):
    # setting record of experiments
    setting = setting_from_args(args, ii)
    
    # set experiments
    exp = Exp(args)
    
    # train
    print(f">>>>>>>start training : {setting}>>>>>>>>>>>>>>>>>>>>>>>>>>")
    exp.train(setting)
    
    # test
    print(f">>>>>>>testing : {setting}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
    exp.test(setting, flag="test")
    exp.test(setting, flag="val")
    exp.test(setting, flag="train")

    torch.cuda.empty_cache()

## Prediction

In [ ]:
# If you already have a trained model, you can set the arguments and model path, then initialize a Experiment and use it to predict
# Prediction is a sequence which is adjacent to the last date of the data, and does not exist in the data
# If you want to get more information about prediction, you can refer to code `exp/exp_informer.py function predict()` and `data/data_loader.py class Dataset_Pred`

manual = False

if manual:
    setting = "informer_custom_ftMS_sl256_ll64_pl16_ei1_di1_co1_iFalse_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_mxTrue_exp_0"
    args = args_from_setting(setting, args)
    exp = Exp(args)

path = os.path.join(args.checkpoints, setting, "checkpoint.pth")

exp.predict(setting, True)

# the prediction will be saved in ./results/{setting}/real_prediction.npy
prediction = np.load(f"./results/{setting}/real_prediction.npy")

print(prediction.shape)

# plt.figure()
# plt.plot(prediction[0,:,-1])
# plt.show()

## Visualization

In [ ]:
# When we finished exp.train(setting) and exp.test(setting), we will get a trained model and the results of test experiment
# The results of test experiment will be saved in ./results/{setting}/pred.npy (prediction of test dataset) and ./results/{setting}/true.npy (groundtruth of test dataset)

tp_dict = {}
for flag in ["train", "val", "test"]:
    preds_path = f"./results/{setting}/pred_{flag}.npy"
    trues_path = f"./results/{setting}/true_{flag}.npy"
    dates_path = f"./results/{setting}/date_{flag}.npy"
    if os.path.exists(preds_path) and os.path.exists(trues_path) and os.path.exists(dates_path):
        tp_dict[flag] = (np.load(trues_path), np.load(preds_path), np.load(dates_path))
        # tp_dict[flag] = list(zip(*sorted(zip(*tp_dict[flag]), key=lambda x: x[-1])))
        s = np.argsort(tp_dict[flag][2], axis=None)
        tp_dict[flag] = list(map(lambda x: x[s], tp_dict[flag]))
        

print("Open true/pred data for:", list(tp_dict.keys()))

# [samples, pred_len, dimensions]
print(tp_dict["train"][0].shape, tp_dict["val"][0].shape, tp_dict["test"][0].shape, "\n\n")

for flag in tp_dict:
    trues, preds, dates = tp_dict[flag]
    print(f"{flag}\ttrues.shape: {trues.shape}, preds.shape: {preds.shape}, dates.shape: {preds.shape}")
    
    MSE = np.square(np.subtract(trues,preds)).mean() 
    RMSE = np.sqrt(MSE)
    print("against preds", MSE, RMSE)

    MSE = np.square(np.subtract(trues,np.zeros(preds.shape))).mean() 
    RMSE = np.sqrt(MSE)
    print("against 0s",MSE,RMSE)

In [ ]:
# draw OT prediction
for flag in tp_dict:
    trues, preds, dates = tp_dict[flag]
    true = trues[:,0,0]
    pred = preds[:,0,0]
    date = dates[:,0]
    plt.figure(num=flag, figsize=(16,4))
    plt.title(flag)
    plt.plot(date, true, label='GroundTruth', linestyle="",marker=".",markersize=4)
    plt.plot(date, pred, label='Prediction',linestyle="",marker=".",markersize=4)
    # plt.scatter(range(trues.shape[0]), trues[:,0,0], marker='v', color='r', label='GroundTruth')
    # plt.scatter(range(trues.shape[0]), preds[:,0,0], marker='^', color='m', label='Prediction')

    plt.legend()
    plt.show()

    plt.figure(num=flag, figsize=(16,4))
    plt.title("Diff histogram")
    # plt.hist(np.abs(true), bins=len(true)//6, label='Diff 0', alpha=0.5)
    # plt.hist(np.abs(true - pred), bins=len(true)//6, label='Diff Pred', alpha=0.5)
    plt.hist([np.abs(true), np.abs(true - pred)], bins=60, label=['Diff 0', 'Diff Pred'])
    plt.xlabel("Diff Value")
    plt.ylabel("Count")
    plt.legend()
    plt.show()

    # df = pd.concat([pd.DataFrame(a, columns=[f"{i}"]) for i, a in enumerate([np.abs(true - pred), np.abs(true)])], axis=1)

    # # plot the data
    # df.plot.hist(stacked=True, bins=len(true), density=True, figsize=(10, 6), grid=True)
    

In [ ]:
m = 0
v = 0

# train percent direction correct, train num excluded,
# val percent direction correct, val num excluded,
# test percent direction correct, test num excluded
tracker = {} 

for f in np.linspace(0, .15, 301):
    # print("f:", f)
    track = {}
    for flag in tp_dict:
        trues, preds = tp_dict[flag]
        true = trues[:,0,0].copy()
        pred = preds[:,0,0].copy()



        
        pred_c = pred[np.abs(pred) >= f]
        true_c = true[np.abs(pred) >= f]
        # print(flag, "num excluded:", len(pred) - len(pred_c), len(pred_c))
        pct_correct = np.sum(np.sign(true_c) == np.sign(pred_c))/len(true_c)
        # print(flag, pct_correct)
        if pct_correct > m and flag=='val':
            m = pct_correct
            v = f
            # print("set",v)
        
        track[f"{flag}_pct_correct"] = pct_correct
        track[f"{flag}_pct_excluded"] = (len(pred) - len(pred_c))/len(pred)

    tracker[f] = track

print("best f", v)
print(tracker[v])



## Attention Visualization

In [ ]:
args.output_attention = True

exp = Exp(args)

model = exp.model

path = os.path.join(args.checkpoints,setting,'checkpoint.pth')

print(model.load_state_dict(torch.load(path)))

df = pd.read_csv(os.path.join(args.root_path, args.data_path))
df[args.cols].head()

In [ ]:
from data_provider.data_loader import Dataset_Custom
from torch.utils.data import DataLoader

Data = Dataset_Custom
timeenc = 0 if args.embed!='timeF' else 1
flag = 'test'; shuffle_flag = False; drop_last = True; batch_size = 1
data_set = Data(args, flag=flag, freq=args.freq, timeenc=timeenc)

data_loader = DataLoader(
    data_set,
    batch_size=batch_size,
    shuffle=shuffle_flag,
    num_workers=args.num_workers,
    drop_last=drop_last)


idx = 0
for i, (batch_x,batch_y,batch_x_mark,batch_y_mark) in enumerate(data_loader):
    if i!=idx:
        continue
    batch_x = batch_x.float().to(exp.device)
    batch_y = batch_y.float()

    batch_x_mark = batch_x_mark.float().to(exp.device)
    batch_y_mark = batch_y_mark.float().to(exp.device)
    
    dec_inp = torch.zeros_like(batch_y[:,-args.pred_len:,:]).float()
    dec_inp = torch.cat([batch_y[:,:args.label_len,:], dec_inp], dim=1).float().to(exp.device)
    
    outputs,attn = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)



print(attn[0].shape, attn[1].shape) #, attn[2].shape



layers = [0,1]
distil = 'Distil' if args.distil else 'NoDistil'
for layer in layers:
    print("\n\n==========================")
    print("Showing attention layer", layer)
    print("==========================\n\n")
    for h in range(0, args.n_heads):
        plt.figure(figsize=[10,8])
        plt.title(f"Informer, {distil}, attn:{args.attn} layer:{layer} head:{h}")
        A = attn[layer][0,h].detach().cpu().numpy()
        ax = sns.heatmap(A, vmin=0, vmax=A.max()+0.01)
        plt.show()